In [7]:
from models import Statements_table_log
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from config import DATABASE_URI
from scrapping_sources.Macrotrend import Macrotrend
import datetime
import pandas as pd

In [8]:
USERNAME='postgres'
HOST='database-webscrap.ccaaerr6cq44.eu-west-3.rds.amazonaws.com'
DATABASE_NAME='webscrapping'
PASSWORD='postgres5678'
PORT=5432
DATABASE_URI = f"postgresql+psycopg2://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/{DATABASE_NAME}"\

session = sessionmaker(bind=create_engine(DATABASE_URI))
s = session()
M = Macrotrend()

In [7]:
def update_statements_log_entry(ticker, stmnt, t_format, period, status):


    s.query(Statements_table_log.__table__).filter(
        Statements_table_log.ticker == ticker,
        Statements_table_log.statement == stmnt,
        Statements_table_log.time_format == t_format,
        Statements_table_log.period == period). \
        update({"status": f"{status}"})
    s.commit()

In [8]:
update_statements_log_entry('MKC', 'income-statement', 'annual', '2021-08', 'test')

In [15]:
def no_table_log(ticker, stmnt, t_format):

    r = s.query(Statements_table_log.__table__).filter(
        Statements_table_log.ticker == ticker,
        Statements_table_log.statement == stmnt,
        Statements_table_log.time_format == t_format,
        Statements_table_log.period == M.latest_ending_period_available(ticker)
    ).all()

    return len(r) == 0

In [16]:
no_table_log("TAYD", 'income-statement', 'annual')

TAYD <Response [200]>


False

In [17]:
s.close_all()

/tmp/ipykernel_7163/1419930820.py:1: SADeprecationWarning: The Session.close_all() method is deprecated and will be removed in a future release.  Please refer to session.close_all_sessions(). (deprecated since: 1.3)
  s.close_all()
